<a href="https://colab.research.google.com/github/satvik-venkatesh/you-only-hear-once/blob/main/YOHO-Music-Speech.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install sed_eval

In [ ]:
!nvidia-smi

In [ ]:
import numpy as np
#import vaex
import tensorflow as tf
#from tensorflow import keras
import IPython
import math
import glob
import sed_eval
import dcase_util
import pickle
import os
import soundfile as sf
import librosa

In [ ]:
os.mkdir("/content/train-zipped")

## The below code downloads data. Each wget command downloads data more than 1GB, so ensure the data downloads correctly. If the downloaded file is very small, please run the code block again 

In [ ]:
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1-6AN2V6E56MID71gjjCMHtDAGkNr97rp' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1-6AN2V6E56MID71gjjCMHtDAGkNr97rp" -O train-zipped/d1.zip && rm -rf /tmp/cookies.txt

In [ ]:
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1-9Z3j41byWsh5boziU5feKs0SksCMBav' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1-9Z3j41byWsh5boziU5feKs0SksCMBav" -O train-zipped/d2.zip && rm -rf /tmp/cookies.txt

In [ ]:
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1-Cn1Gnwls8KoZb19-7eFWO8MvKM-u7T2' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1-Cn1Gnwls8KoZb19-7eFWO8MvKM-u7T2" -O train-zipped/d3.zip && rm -rf /tmp/cookies.txt

In [ ]:
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1-E51YUmfOu9828Cgy2bMYZm8dSHuHnqr' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1-E51YUmfOu9828Cgy2bMYZm8dSHuHnqr" -O train-zipped/d4.zip && rm -rf /tmp/cookies.txt

In [ ]:
"""
Extract the .zip files into the 'train data' folder.
"""
from zipfile import ZipFile

for i in range(0, 4):
  zip_name = "/content/train-zipped/d" + str(i + 1) + ".zip"
  with ZipFile(zip_name, 'r') as zip:
    zip.extractall('train data')
    print("Extracted all sound files into the folder {}".format(i + 1))

In [ ]:
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1-xOXLHq55-J4f700oKbEDHFuLxiavit2' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1-xOXLHq55-J4f700oKbEDHFuLxiavit2" -O train-zipped/BBC-Train.zip && rm -rf /tmp/cookies.txt

In [ ]:
"""
Extracting Train data
"""
from zipfile import ZipFile
zip_name = "/content/train-zipped/BBC-Train.zip"
with ZipFile(zip_name, 'r') as zip:
  zip.extractall('train data')
  print("Extracted all sound files into the folder")


In [ ]:
os.mkdir("/content/val-zipped")

In [ ]:
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1-y8XbsH5ou5vYSptB7QnhWByj0rkvSol' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1-y8XbsH5ou5vYSptB7QnhWByj0rkvSol" -O val-zipped/BBC-Val.zip && rm -rf /tmp/cookies.txt

In [ ]:
"""
Extract the .zip files into the 'val data' folder.
"""
from zipfile import ZipFile

zip_name = "/content/val-zipped/BBC-Val.zip"
with ZipFile(zip_name, 'r') as zip:
  zip.extractall('validation data')
  print("Extracted all sound files into the folder {}".format(i + 1))

In [ ]:
def smoothe_events(events):
  music_events = []
  speech_events = []
  for e in events:
    if e[2] == "speech":
      speech_events.append(e)
    elif e[2] == "music":
      music_events.append(e)

  speech_events.sort(key=lambda x: x[0])
  music_events.sort(key=lambda x: x[0])


  start_speech = -1000
  stop_speech = -1000

  speech_events_2 = []

  max_speech_silence = 0.4
  max_music_silence = 0.6
  min_dur_speech = 1.3
  min_dur_music = 3.4

  # max_speech_silence = 0.0
  # max_music_silence = 0.0
  # min_dur_speech = 0.0
  # min_dur_music = 0.0
  count = 0

  while count < len(speech_events) - 1:
    if (speech_events[count][1] >= speech_events[count + 1][0]) or (speech_events[count + 1][0] - speech_events[count][1] <= max_speech_silence):
      speech_events[count][1] = max(speech_events[count + 1][1], speech_events[count][1])
      del speech_events[count + 1]
    else:
      count += 1

  count = 0

  while count < len(music_events) - 1:
    if (music_events[count][1] >= music_events[count + 1][0]) or (music_events[count + 1][0] - music_events[count][1] <= max_music_silence):
      music_events[count][1] = max(music_events[count + 1][1], music_events[count][1])
      del music_events[count + 1]
    else:
      count += 1


  smooth_events = music_events + speech_events


  # count = 0
  # while count < len(smooth_events):
  #   if smooth_events[count][1] - smooth_events[count][0] < min_dur_speech and smooth_events[count][2] == "speech":
  #     del smooth_events[count]

  #   elif smooth_events[count][1] - smooth_events[count][0] < min_dur_music and smooth_events[count][2] == "music":
  #     del smooth_events[count]

  #   else:
  #     count += 1

  for i in range(len(smooth_events)):
    smooth_events[i][0] = round(smooth_events[i][0], 3)
    smooth_events[i][1] = round(smooth_events[i][1], 3)

  smooth_events.sort(key=lambda x: x[0])

  return smooth_events

In [ ]:
def get_universal_labels(events, no_of_div = 26):
  """Generate the label matrix of an audio sample based on its annotations."""
  events = smoothe_events(events)
  win_length = 8.0/no_of_div
  labels = np.zeros((no_of_div, 6))

  for e in events:

    start_time = float(e[0])
    stop_time = float(e[1])

    start_bin = int(start_time // win_length)
    stop_bin = int(stop_time // win_length)

    start_time_2 = start_time - start_bin * win_length
    stop_time_2 = stop_time - stop_bin * win_length

    n_bins = stop_bin - start_bin

    if n_bins == 0:
      if e[2] == "speech":
        labels[start_bin, 0:3] = [1, start_time_2, stop_time_2]

      elif e[2] == "music":
        labels[start_bin, 3:6] = [1, start_time_2, stop_time_2]

    elif n_bins == 1:
      if e[2] == "speech":
        labels[start_bin, 0:3] = [1, start_time_2, win_length]

      elif e[2] == "music":
        labels[start_bin, 3:6] = [1, start_time_2, win_length]

      if stop_time_2 > 0.0:
        if e[2] == "speech":
          labels[stop_bin, 0:3] = [1, 0.0, stop_time_2]

        elif e[2] == "music":
          labels[stop_bin, 3:6] = [1, 0.0, stop_time_2]

    elif n_bins > 1:
      if e[2] == "speech":
        labels[start_bin, 0:3] = [1, start_time_2, win_length]

      if e[2] == "music":
        labels[start_bin, 3:6] = [1, start_time_2, win_length]

      for i in range(1, n_bins):
        if e[2] == "speech":
          labels[start_bin + i, 0:3] = [1, 0.0, win_length]

        if e[2] == "music":
          labels[start_bin + i, 3:6] = [1, 0.0, win_length]

      if stop_time_2 > 0.0:
        if e[2] == "speech":
          labels[stop_bin, 0:3] = [1, 0.0, stop_time_2]

        elif e[2] == "music":
          labels[stop_bin, 3:6] = [1, 0.0, stop_time_2]

  labels[:, [1, 2, 4, 5]] /= win_length

  return labels

In [ ]:
def get_only_binary_labels(events, win_length = 8.0/13):
  """Generate the label matrix of an audio sample based on its annotations."""
  events = smoothe_events(events)
  labels = np.zeros((2,))

  class_list = [e[2] for e in events]

  if "speech" in class_list:
    labels[0] = 1

  if "music" in class_list:
    labels[1] = 1

  return labels

In [ ]:
def construct_labels(labels, no_of_steps = 4):
  # labels = smoothe_events(labels)

  new_labels = np.zeros((no_of_steps, 6))
  win_width = 8.0 / no_of_steps


  for i in range(len(labels)):
    s = labels[i][0] / win_width
    s = min(np.floor(s), no_of_steps - 1)

    r = s * win_width

    if labels[i][2] == "speech":
      new_labels[int(s)][0] = 1

      t1 = (labels[i][0] - r) / win_width
      t2 = (labels[i][1] - labels[i][0]) / win_width

      new_labels[int(s)][1] = t1
      new_labels[int(s)][2] = t2

    elif labels[i][2] == "music":
      new_labels[int(s)][3] = 1

      t1 = (labels[i][0] - r) / win_width
      t2 = (labels[i][1] - labels[i][0]) / win_width

      new_labels[int(s)][4] = t1
      new_labels[int(s)][5] = t2

  return new_labels

In [ ]:
def to_seg_by_class(events, n_frames = 801):
  events = smoothe_events(events)
  labels = np.zeros((n_frames, 2), dtype=np.float32)

  for e in events:
    t1 = float(e[0])
    t1 = int(t1 / 160 * 16000)
    t2 = float(e[1])
    t2 = int(t2 / 160 * 16000)

    if e[2] == 'speech':
      labels[t1:t2, 0] = 1
    elif e[2] == 'music':
      labels[t1:t2, 1] = 1
  
  return labels 

In [ ]:
events = [[0.2, 4.3, "music"], [3.6, 6.0, "speech"]]
ss = get_universal_labels(events)
print(ss)

In [ ]:
def get_log_melspectrogram(audio, sr = 16000, hop_length = 160, win_length = 400, n_fft = 512, n_mels = 64, fmin = 125, fmax = 7500):
    """Return the log-scaled Mel bands of an audio signal."""
    bands = librosa.feature.melspectrogram(
        y=audio, sr=sr, hop_length=hop_length, win_length = win_length, n_fft=n_fft, n_mels=n_mels, fmin=fmin, fmax=fmax, dtype=np.float32)
    return librosa.core.power_to_db(bands, amin=1e-7)

In [ ]:
labels = glob.glob("/content/train data/content/**/mel-id-label-[0-9]*.pickle", recursive=True)

for ll in labels:
  with open(ll, 'rb') as f:
    n = pickle.load(f)
  n2 = get_universal_labels(n)
  np.save(ll.replace(".pickle", ".npy"), n2)


labels = glob.glob("/content/validation data/**/mel-id-label-[0-9]*.pickle", recursive=True)

for ll in labels:
  with open(ll, 'rb') as f:
    n = pickle.load(f)
  n2 = get_universal_labels(n)
  np.save(ll.replace(".pickle", ".npy"), n2)

In [ ]:
import tensorflow as tf
import keras

class DataGenerator(tf.keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self, list_examples, batch_size=128, epoch_size = 16384, dim=(1, ),
                 n_classes=2, shuffle=True):
        'Initialization'
        print("Constructor called!!!")
        self.dim = dim
        self.batch_size = batch_size
        self.epoch_size = epoch_size
        self.list_examples = list_examples
        self.n_classes = n_classes
        self.shuffle = shuffle
        self.on_epoch_end()

    def __len__(self):
        'Denotes the number of batches per epoch'
        #print("The self.list_examples is {}".format(self.list_examples))
        return int(np.floor(len(self.list_examples) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

        # Find list of IDs
        list_IDs_temp = [self.list_examples[k] for k in indexes]

        # Generate data
        X, y = self.__data_generation(list_IDs_temp)

        return X, y

    def on_epoch_end(self):
      self.indexes = np.arange(len(self.list_examples))
      if self.shuffle == True:
          np.random.shuffle(self.indexes)

    def __data_generation(self, list_IDs_temp):
        # 'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
        # # Initialization
        X = np.empty([self.batch_size, 801, 64, 1], dtype=np.float64)
        y = np.empty([self.batch_size, 26, 6], dtype=np.float64)

        # Generate data
        for i, ID in enumerate(list_IDs_temp):
          # Store sample

          xx = np.load(ID[0])
          X[i, :, :, 0] = xx

          # Store class
          yy = np.load(ID[1])
          # yy2 = yy[:, [1, 2, 4, 5]]
          y[i, :, :] = yy

        # v = X.shape[0]
        # tau = X.shape[1]

        # warped_frequency_spectrogram = spec_augment_tensorflow.frequency_masking(X, v=v)
        # warped_frequency_time_sepctrogram = spec_augment_tensorflow.time_masking(warped_frequency_spectrogram, tau=tau)

        # X = warped_frequency_time_sepctrogram

        return X, y

In [ ]:
import re

def tryint(s):
    try:
        return int(s)
    except ValueError:
        return s
    
def alphanum_key(s):
    """ Turn a string into a list of string and number chunks.
        "z23a" -> ["z", 23, "a"]
    """
    return [ tryint(c) for c in re.split('([0-9]+)', s) ]

def sort_nicely(l):
    """ Sort the given list in the way that humans expect.
    """
    l.sort(key=alphanum_key)

In [ ]:
import glob
import random
"""
Load the individual numpy arrays into partition
"""
data = glob.glob("/content/train data/content/**/mel-id-[0-9]*.npy", recursive=True) 
sort_nicely(data)

labels = glob.glob("/content/train data/content/**/mel-id-label-[0-9]*.npy", recursive=True)
sort_nicely(labels)

train_examples = [(data[i], labels[i]) for i in range(len(data))]

random.seed(4)
random.shuffle(train_examples)

In [ ]:
len(train_examples)

In [ ]:
"""
Creating the train partition.
"""
m_train = 25184
random.seed()
random.shuffle(train_examples)

data_MS = glob.glob("/content/train data/MuSpeak/content/Mel Files/**/mel-id-[0-9]*.npy", recursive=True) 
sort_nicely(data_MS)

labels_MS = glob.glob("/content/train data/MuSpeak/content/Mel Files/**/mel-id-label-[0-9]*.npy", recursive=True)
sort_nicely(labels_MS)

train_examples_MS = [(data_MS[i], labels_MS[i]) for i in range(len(data_MS))]

partition = {}
partition['train'] = train_examples[0:m_train] + train_examples_MS

random.shuffle(partition['train'])

In [ ]:
print("The size of partition['train'] is {}".format(len(partition['train'])))

In [ ]:
"""
This loads data for the validation set.
"""
import glob
import random

data = glob.glob("/content/validation data/**/mel-id-[0-9]*.npy", recursive=True)
sort_nicely(data)

labels = glob.glob("/content/validation data/**/mel-id-label-[0-9]*.npy", recursive=True)
sort_nicely(labels)

validation_examples = [(data[i], labels[i]) for i in range(len(data))]

random.seed(4)
random.shuffle(validation_examples)
print(validation_examples[0])

# m = len(test_examples)
# m_validation = 1024
# m_test = 512
# m_train = m - m_validation - m_test

partition['validation'] = validation_examples

In [ ]:
len(partition['train'])

In [ ]:
# Parametersa
params = {'dim': (1, ),
          'batch_size': 32,
          'epoch_size': 0,
          'n_classes': 2,
          'shuffle': True}

# Generators
training_generator = DataGenerator(partition['train'], **params)
validation_generator = DataGenerator(partition['validation'], **params)

In [ ]:
"""
This function was adapted from Lemaire et al. 2019. 
"""

def smooth_output(output, min_speech=1.3, min_music=3.4, max_silence_speech=0.4, max_silence_music=0.6):
    duration_frame = 220 / 22050
    n_frame = output.shape[1]

    start_music = -1000
    start_speech = -1000

    for i in range(n_frame):
        if output[0, i] == 1:
            if i - start_speech > 1:
                if (i - start_speech) * duration_frame <= max_silence_speech:
                    output[0, start_speech:i] = 1

            start_speech = i

        if output[1, i] == 1:
            if i - start_music > 1:
                if (i - start_music) * duration_frame <= max_silence_music:
                    output[1, start_music:i] = 1

            start_music = i

    start_music = -1000
    start_speech = -1000

    for i in range(n_frame):
        if i != n_frame - 1:
            if output[0, i] == 0:
                if i - start_speech > 1:
                    if (i - start_speech) * duration_frame <= min_speech:
                        output[0, start_speech:i] = 0

                start_speech = i

            if output[1, i] == 0:
                if i - start_music > 1:
                    if (i - start_music) * duration_frame <= min_music:
                        output[1, start_music:i] = 0

                start_music = i
        else:
            if i - start_speech > 1:
                if (i - start_speech) * duration_frame <= min_speech:
                    output[0, start_speech:i + 1] = 0

            if i - start_music > 1:
                if (i - start_music) * duration_frame <= min_music:
                    output[1, start_music:i + 1] = 0

    return output

"""
This function converts the predictions made by the neural network into a format that is understood by sed_eval.
"""

def preds_to_se(p, audio_clip_length = 8.0):
  start_speech = -100
  start_music = -100
  stop_speech = -100
  stop_music = -100

  audio_events = []

  n_frames = p.shape[0]

  if p[0, 0] == 1:
    start_speech = 0
  
  if p[0, 1] == 1:
    start_music = 0

  for i in range(n_frames - 1):
    if p[i, 0] == 0 and p[i + 1, 0] == 1:
      start_speech = i + 1

    elif p[i, 0] == 1 and p[i + 1, 0] == 0:
      stop_speech = i
      start_time = frames_to_time(start_speech)
      stop_time = frames_to_time(stop_speech)
      audio_events.append((start_time, stop_time, "speech"))
      start_speech = -100
      stop_speech = -100

    if p[i, 1] == 0 and p[i + 1, 1] == 1:
      start_music = i + 1
    elif p[i, 1] == 1 and p[i + 1, 1] == 0:
      stop_music = i
      start_time = frames_to_time(start_music)
      stop_time = frames_to_time(stop_music)      
      audio_events.append((start_time, stop_time, "music"))
      start_music = -100
      stop_music = -100

  if start_speech != -100:
    start_time = frames_to_time(start_speech)
    stop_time = audio_clip_length
    audio_events.append((start_time, stop_time, "speech"))
    start_speech = -100
    stop_speech = -100

  if start_music != -100:
    start_time = frames_to_time(start_music)
    stop_time = audio_clip_length
    audio_events.append((start_time, stop_time, "music"))
    start_music = -100
    stop_music = -100

  audio_events.sort(key = lambda x: x[0]) 
  return audio_events

def frames_to_time(f, sr = 22050.0, hop_size = 220):
  return f * hop_size / sr

def get_log_melspectrogram(audio, sr = 16000, hop_length = 160, win_length = 400, n_fft = 512, n_mels = 64, fmin = 125, fmax = 7500):
    """Return the log-scaled Mel bands of an audio signal."""
    bands = librosa.feature.melspectrogram(
        y=audio, sr=sr, hop_length=hop_length, win_length = win_length, n_fft=n_fft, n_mels=n_mels, fmin=fmin, fmax=fmax, dtype=np.float32)
    return librosa.core.power_to_db(bands, amin=1e-7)


"""
Make predictions for full audio.
"""
def mk_preds_fa(audio_path, hop_size = 6.0, discard = 1.0, win_length = 8.0, sampling_rate = 22050):
  in_signal, in_sr = sf.read(audio_path)

  # Resample the audio file.
  in_signal_22k = librosa.resample(in_signal, orig_sr=in_sr, target_sr=sampling_rate)
  in_signal = np.copy(in_signal_22k)

  audio_clip_length_samples = in_signal.shape[0]
  print('audio_clip_length_samples is {}'.format(audio_clip_length_samples))

  #hop_size_samples = int(hop_size * sampling_rate)
  hop_size_samples = 220 * 602 - 1

  #win_length_samples = int(win_length * sampling_rate)
  win_length_samples = 220 * 802 - 1

  n_preds = int(math.ceil((audio_clip_length_samples - win_length_samples) / hop_size_samples)) + 1

  #print('n_preds is {}'.format(n_preds))

  in_signal_pad = np.zeros((n_preds * hop_size_samples + 200 * 220))

  #print('in_signal_pad.shape is {}'.format(in_signal_pad.shape))

  in_signal_pad[0:audio_clip_length_samples] = in_signal

  preds = np.zeros((n_preds, 802, 2))
  mss_in = np.zeros((n_preds, 802, 80))

  for i in range(n_preds):
    seg = in_signal_pad[i * hop_size_samples:(i * hop_size_samples) + win_length_samples]
    #print('seg.shape is {}'.format(seg.shape))
    seg = librosa.util.normalize(seg)

    mss = get_log_melspectrogram(seg)
    M = mss.T
    mss_in[i, :, :] = M

  preds = (model.predict(mss_in) >= (0.5, 0.5)).astype(np.float)
  #p = cat_to_multi(p)

  # preds[i, :, :] = p[0]

  #discard_frames = 100
  #oa_preds = np.zeros((, 128)) # overall predictions

  preds_mid = np.copy(preds[1:-1, 100:702, :])

  #print("preds_mid.shape is {}".format(preds_mid.shape))

  preds_mid_2 = preds_mid.reshape(-1, 2)

  oa_preds = preds[0, 0:702, :] # oa stands for overall predictions

  oa_preds = np.concatenate((oa_preds, preds_mid_2), axis = 0)

  oa_preds = np.concatenate((oa_preds, preds[-1, 100:, :]), axis = 0)

  return oa_preds



In [ ]:
def my_loss_fn(y_true, y_pred):
  weight = tf.constant([1.0])
  squared_difference = tf.square(y_true - y_pred)

  # squared_difference[:, 1] =  squared_difference[:, 1] * y_true[:, 0]

  # squared_difference[:, 3] =  squared_difference[:, 3] * y_true[:, 2]
  ss0 = squared_difference[:, :, 0] * 0 + 1
  ss1 = y_true[:, :, 0] 
  ss2 = y_true[:, :, 0]

  ss3 = squared_difference[:, :, 3] * 0 + 1
  ss4 = y_true[:, :, 3] 
  ss5 = y_true[:, :, 3]



  sss = tf.stack((ss0, ss1, ss2, ss3, ss4, ss5), axis = 2)
  
  squared_difference =  tf.multiply(squared_difference, sss)


  return tf.reduce_sum(squared_difference, axis=[-1, -2])  # Note the `axis=-1`

In [ ]:
def binary_acc(y_true, y_pred):
  threshold = tf.constant([0.5])

  binary_true = tf.stack((y_true[:, :, 0], y_true[:, :, 3]), axis=1)
  binary_pred = tf.stack((y_pred[:, :, 0], y_pred[:, :, 3]), axis=1)

  binary_true = tf.greater_equal(binary_true, threshold)
  binary_pred = tf.greater_equal(binary_pred, threshold)

  # binary_pred = tf.greater(y_pred[:, [0, 2]], threshold)

  # acc = tf.square(y_true - y_pred)
  acc = tf.cast((binary_true == binary_pred), tf.float32)

  return tf.reduce_mean(acc, axis=[-1, -2])  # Note the `axis=-1`

In [ ]:
class TimeAcc(tf.keras.metrics.Metric):

  def __init__(self, name='time_accuracy', **kwargs):
    super(TimeAcc, self).__init__(name=name, **kwargs)
    self.time_correct_1 = self.add_weight(name='time_correct_1', initializer='zeros')
    self.time_count_1 = self.add_weight(name='time_count_1', initializer='zeros')
    self.time_correct_3 = self.add_weight(name='time_correct_3', initializer='zeros')
    self.time_count_3 = self.add_weight(name='time_count_3', initializer='zeros')

  def update_state(self, y_true, y_pred, sample_weight=None):
    tolerance = tf.constant([0.25])
    zero_lim = tf.constant([0.0])

    y_true_0 = y_true[:, :, 0]
    y_true_1 = y_true[:, :, 1]

    y_pred_0 = y_pred[:, :, 0]
    y_pred_1 = y_pred[:, :, 1]

    time_diff_1 = tf.abs(y_pred_1 - y_true_1)

    time_correct_1 = tf.cast(tf.less_equal(time_diff_1, tolerance), dtype = np.float32)

    time_correct_1 = tf.multiply(time_correct_1, y_true_0)

    self.time_count_1.assign_add(tf.reduce_sum(y_true_0, axis=None))

    self.time_correct_1.assign_add(tf.reduce_sum(time_correct_1, axis = None))

  def result(self):
    time_acc_1 = self.time_correct_1 / self.time_count_1
    return time_acc_1

  def reset_states(self):
    self.time_correct_1.assign(0)
    self.time_count_1.assign(0)

In [ ]:
initial_learning_rate = 0.001
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate,
    decay_steps=2500,
    decay_rate=0.84,
    staircase=True)

In [ ]:
# This optimises val loss for Wave-U-Net YOHO
# Back to Val Binary acc.

import os
class MyCustomCallback_3(tf.keras.callbacks.Callback):
  def __init__(self, model_dir, patience=0):
    super(MyCustomCallback_3, self).__init__()
    self.patience = patience
    # best_weights to store the weights at which the minimum loss occurs.
    self.best_weights = None
    self.model_best_path = os.path.join(model_dir, 'model-best.h5')
    self.model_last_path = os.path.join(model_dir, 'model-last-epoch.h5')
    self.custom_params = {"best_loss":np.inf, "last_epoch":0}
    
    self.custom_params_path = os.path.join(model_dir, 'custom_params.pickle')
    if os.path.isfile(self.custom_params_path):
      with open(self.custom_params_path, 'rb') as f:
        self.custom_params = pickle.load(f)
      # best_model = tf.keras.models.load_model(self.model_best_path, custom_objects={'my_loss_fn':my_loss_fn, 
      #             'binary_acc':binary_acc, 'TCN':TCN(), 'SpeechF1':SpeechF1(), 'MusicF1':MusicF1(),
      #             'OnAcc':OnAcc(), 'OnOffAcc':OnOffAcc(),'AudioClipLayer':AudioClipLayer(), 
      #             'InterpolationLayer':InterpolationLayer(), 'CropLayer':CropLayer(),
      #             'IndependentOutputLayer':IndependentOutputLayer(), 'DiffOutputLayer':DiffOutputLayer()})
      # best_model = tf.keras.models.clone_model(self.model)
      # best_model.load_weights(self.model_best_path)
      # self.best_weights = best_model.get_weights()


  def on_train_begin(self, logs=None):
    # The number of epoch it has waited when loss is no longer minimum.
    self.wait = 0
    # The epoch the training stops at.
    self.stopped_epoch = 0
    # Initialize the best F1 as 0.0.
    self.is_impatient = False

  def on_train_end(self, logs=None):
    if not self.is_impatient:
      print("Restoring model weights from the end of the best epoch.")
      self.model.set_weights(self.best_weights)
      # temp_model_path = self.model_path.replace(".h5", "_temp.h5")
      #os.remove(temp_model_path)

  def on_epoch_end(self, epoch, logs=None):
    current_val_loss = logs.get("val_loss")
    self.model.save_weights(self.model_last_path)
    self.custom_params["last_epoch"] = self.custom_params["last_epoch"] + 1

    if current_val_loss < self.custom_params['best_loss']:
      self.custom_params['best_loss'] = current_val_loss
      self.wait = 0
      self.best_weights = self.model.get_weights()
      self.model.save_weights(self.model_best_path)

    else:
        self.wait += 1
        if self.wait >= self.patience:
            self.stopped_epoch = epoch
            self.is_impatient = True
            self.model.stop_training = True
            print("Restoring model weights from the end of the best epoch.")
            self.model.set_weights(self.best_weights)
            #os.remove(temp_model_path)
    with open(self.custom_params_path, 'wb') as f:
      pickle.dump(self.custom_params, f, pickle.HIGHEST_PROTOCOL)

# The YOHO network

In [ ]:
import tensorflow as tf

In [ ]:
LAYER_DEFS = [
    # (layer_function, kernel, stride, num_filters)
    ([3, 3], 1,   64),
    ([3, 3], 2,  128),
    ([3, 3], 1,  128),
    ([3, 3], 2,  256),
    ([3, 3], 1,  256),
    ([3, 3], 2,  512),
    ([3, 3], 1,  512),
    ([3, 3], 1,  512),
    ([3, 3], 1,  512),
    ([3, 3], 1,  512),
    ([3, 3], 1,  512),
    ([3, 3], 2, 1024),
    ([3, 3], 1, 1024),
    ([3, 3], 1, 512),
    ([3, 3], 1, 256),
    ([3, 3], 1, 128),
    # ([3, 3], 1, 128),
    # ([3, 3], 1, 128)
]

In [ ]:
"""
Manually define YOHO
"""

# params = yamnet_params.Params()
m_features = tf.keras.Input(shape=(801, 64, 1), name="mel_input")
X = m_features
# X = tf.keras.layers.Reshape((801, 64, 1))(X)
X = tf.keras.layers.Conv2D(filters = 32, kernel_size=[3, 3], strides=2, padding='same', use_bias=False, activation=None, name = "layer1/conv")(X)
X = tf.keras.layers.BatchNormalization(center=True, scale=False, epsilon=1e-4, name = "layer1/bn")(X)
X = tf.keras.layers.ReLU(name="layer1/relu")(X)

for i in range(len(LAYER_DEFS)):
  X = tf.keras.layers.DepthwiseConv2D(kernel_size=LAYER_DEFS[i][0], strides = LAYER_DEFS[i][1], depth_multiplier=1, padding='same', use_bias=False,
                                      activation=None, name="layer"+ str(i + 2)+"/depthwise_conv")(X)
  X = tf.keras.layers.BatchNormalization(center=True, scale=False, epsilon=1e-4, name = "layer"+ str(i + 2)+"/depthwise_conv/bn")(X)
  X = tf.keras.layers.ReLU(name="layer"+ str(i + 2)+"/depthwise_conv/relu")(X)
  X = tf.keras.layers.Conv2D(filters = LAYER_DEFS[i][2], kernel_size=[1, 1], strides=1, padding='same', use_bias=False, activation=None,
                             name = "layer"+ str(i + 2)+"/pointwise_conv")(X)
  X = tf.keras.layers.BatchNormalization(center=True, scale=False, epsilon=1e-4, name = "layer"+ str(i + 2)+"/pointwise_conv/bn")(X)
  X = tf.keras.layers.ReLU(name="layer"+ str(i + 2)+"/pointwise_conv/relu")(X)


_, _, sx, sy = X.shape
X = tf.keras.layers.Reshape((-1, int(sx * sy)))(X)

pred = tf.keras.layers.Conv1D(6,kernel_size=1, activation="sigmoid")(X)
model = tf.keras.Model(
      name='yamnet_frames', inputs=m_features,
      outputs=[pred])

In [ ]:
model.summary()

In [ ]:
"""
BInary accuracy is not a valid measure for the YOHO algorithm. However, it gives a rough idea of how the model is doing because all the values are normalized between 0 and 1.
The metrics are implemented in the testing section and val loss is used for early stopping.
"""

model.compile(optimizer=tf.keras.optimizers.Adam(1e-3), loss=my_loss_fn, metrics=[binary_acc])

In [ ]:
# """
# The CRNN model used for ICASSP
# """

# import tensorflow as tf
# from tensorflow import keras
# from tensorflow.keras import layers

# mel_input = keras.Input(shape=(801, 64, 1), name="mel_input")
# X = mel_input

# # X = tf.keras.layers.Reshape((801, 64, 1))(X)
# # print(X.shape)

# X = tf.keras.layers.Conv2D(filters=16, kernel_size=7, strides=1, padding='same')(X)
# X = layers.BatchNormalization(momentum=0.0)(X)
# X = tf.keras.layers.Activation('relu')(X)
# X = tf.keras.layers.MaxPool2D(pool_size=(1, 2))(X)
# X = tf.keras.layers.Dropout(rate = 0.2)(X)

# X = tf.keras.layers.Conv2D(filters=64, kernel_size=7, strides=1, padding='same')(X)
# X = layers.BatchNormalization(momentum=0.0)(X)
# X = tf.keras.layers.Activation('relu')(X)
# X = tf.keras.layers.MaxPool2D(pool_size=(1, 2))(X)
# X = layers.Dropout(rate = 0.2)(X)

# print(X.shape)
# _, _, sx, sy = X.shape
# X = tf.keras.layers.Reshape((-1, int(sx * sy)))(X)

# X = layers.Bidirectional(layers.GRU(80, return_sequences = True))(X)
# X = layers.BatchNormalization(momentum=0.0)(X)

# X = layers.Bidirectional(layers.GRU(80, return_sequences = True))(X)
# X = layers.BatchNormalization(momentum=0.0)(X)

# pred = layers.TimeDistributed(layers.Dense(2, activation='sigmoid'))(X)

# model = keras.Model(inputs = [mel_input], outputs = [pred])

# keras.utils.plot_model(model, "CRNN.png", show_shapes=True)

# model.compile(
#     optimizer=keras.optimizers.Adam(learning_rate=0.001),
#     loss=[keras.losses.BinaryCrossentropy()], metrics=['binary_accuracy'] #, 'categorical_accuracy', tf.keras.metrics.Precision(class_id=0), tf.keras.metrics.Precision(class_id=1), tf.keras.metrics.Recall(class_id=0), tf.keras.metrics.Recall(class_id=1)]
# )

# model.summary()

In [ ]:
# model.compile(optimizer=tf.keras.optimizers.Adam(1e-4), loss=my_loss_fn, metrics=[binary_acc])

In [ ]:
# Load the TensorBoard notebook extension
root_dir = "/content/Models"
model_name = 'YOHO-1'
model_dir = os.path.join(root_dir, model_name)

try: 
    os.mkdir(model_dir) 
except OSError as error: 
    pass  

%load_ext tensorboard
import datetime, os
logdir = os.path.join(root_dir)
tensorboard_callback = tf.keras.callbacks.TensorBoard(os.path.join(root_dir, model_name), histogram_freq=1)

%tensorboard --logdir "{logdir}"

In [ ]:
initial_epoch = 0
p = os.path.join(model_dir, 'custom_params.pickle')
if os.path.isfile(p):
  print("Existing model found. Loading weights and training ...")
  with open(p, 'rb') as f:
    custom_params = pickle.load(f)
    last_epoch = custom_params['last_epoch']
    initial_epoch = last_epoch
  model_path = os.path.join(model_dir, 'model-best.h5')
  print(model_path)
  model.load_weights(model_path)

  # model.load_weights(model_path)
  model.fit(training_generator, validation_data=validation_generator, epochs=300, initial_epoch=initial_epoch,
            callbacks=[MyCustomCallback_3(model_dir, patience=15), tensorboard_callback], verbose=1)
  
else:
  print("No existing model found. Begin training ...")

  model.fit(training_generator, validation_data=validation_generator, epochs=300,
            callbacks=[MyCustomCallback_3(model_dir, patience=15), tensorboard_callback], verbose=1)
  
  # workers=1,
  #   use_multiprocessing=False,

# If you encounter an error in the above code block, please check if the initial 'wget' blocks were executed properly and the data was downloaded.

In [ ]:
model.load_weights("/content/Models/YOHO-1/model-best.h5")

In [ ]:
model.evaluate(validation_generator)

Please note that binary accuracy is not a valid measure for the YOHO algorithm. However, it gives a rough idea of how the model is doing because all the values are normalized between 0 and 1.
The metrics for F1 are implemented in the testing section.

In [ ]:
"""
An evaluation when training was complete
"""
model.evaluate(validation_generator)

In [ ]:
!nvidia-smi

# Testing

In [ ]:
def smooth_output(output, min_speech=1.3, min_music=3.4, max_silence_speech=0.4, max_silence_music=0.6):
    duration_frame = 220 / 22050
    n_frame = output.shape[1]

    start_music = -1000
    start_speech = -1000

    for i in range(n_frame):
        if output[0, i] == 1:
            if i - start_speech > 1:
                if (i - start_speech) * duration_frame <= max_silence_speech:
                    output[0, start_speech:i] = 1

            start_speech = i

        if output[1, i] == 1:
            if i - start_music > 1:
                if (i - start_music) * duration_frame <= max_silence_music:
                    output[1, start_music:i] = 1

            start_music = i

    start_music = -1000
    start_speech = -1000

    for i in range(n_frame):
        if i != n_frame - 1:
            if output[0, i] == 0:
                if i - start_speech > 1:
                    if (i - start_speech) * duration_frame <= min_speech:
                        output[0, start_speech:i] = 0

                start_speech = i

            if output[1, i] == 0:
                if i - start_music > 1:
                    if (i - start_music) * duration_frame <= min_music:
                        output[1, start_music:i] = 0

                start_music = i
        else:
            if i - start_speech > 1:
                if (i - start_speech) * duration_frame <= min_speech:
                    output[0, start_speech:i + 1] = 0

            if i - start_music > 1:
                if (i - start_music) * duration_frame <= min_music:
                    output[1, start_music:i + 1] = 0

    return output

In [ ]:
def smoothe_events(events):
  music_events = []
  speech_events = []
  for e in events:
    if e[2] == "speech":
      speech_events.append(e)
    elif e[2] == "music":
      music_events.append(e)

  speech_events.sort(key=lambda x: x[0])
  music_events.sort(key=lambda x: x[0])


  start_speech = -1000
  stop_speech = -1000

  speech_events_2 = []

  max_speech_silence = 0.8
  max_music_silence = 0.8
  min_dur_speech = 0.8
  min_dur_music = 3.4

  count = 0

  while count < len(speech_events) - 1:
    if (speech_events[count][1] >= speech_events[count + 1][0]) or (speech_events[count + 1][0] - speech_events[count][1] <= max_speech_silence):
      speech_events[count][1] = max(speech_events[count + 1][1], speech_events[count][1])
      del speech_events[count + 1]
    else:
      count += 1

  count = 0

  while count < len(music_events) - 1:
    if (music_events[count][1] >= music_events[count + 1][0]) or (music_events[count + 1][0] - music_events[count][1] <= max_music_silence):
      music_events[count][1] = max(music_events[count + 1][1], music_events[count][1])
      del music_events[count + 1]
    else:
      count += 1


  smooth_events = music_events + speech_events


  count = 0
  while count < len(smooth_events):
    if smooth_events[count][1] - smooth_events[count][0] < min_dur_speech and smooth_events[count][2] == "speech":
      del smooth_events[count]

    elif smooth_events[count][1] - smooth_events[count][0] < min_dur_music and smooth_events[count][2] == "music":
      del smooth_events[count]

    else:
      count += 1

  for i in range(len(smooth_events)):
    smooth_events[i][0] = round(smooth_events[i][0], 3)
    smooth_events[i][1] = round(smooth_events[i][1], 3)

  smooth_events.sort(key=lambda x: x[0])

  return smooth_events

In [ ]:
"""
Make predictions for full audio --- vectorised implementation.
"""
def mk_preds_vector(audio_path, hop_size = 6.0, discard = 1.0, win_length = 8.0, sampling_rate = 22050):
  in_signal, in_sr = sf.read(audio_path)

  # Resample the audio file.
  in_signal_22k = librosa.resample(in_signal, orig_sr=in_sr, target_sr=sampling_rate)
  in_signal = np.copy(in_signal_22k)

  audio_clip_length_samples = in_signal.shape[0]
  print('audio_clip_length_samples is {}'.format(audio_clip_length_samples))

  hop_size_samples = int(hop_size * sampling_rate)
  # hop_size_samples = 220 * 602 - 1

  win_length_samples = int(win_length * sampling_rate)
  # win_length_samples = 220 * 802 - 1

  n_preds = int(math.ceil((audio_clip_length_samples - win_length_samples) / hop_size_samples)) + 1

  # n_preds = int()

  #print('n_preds is {}'.format(n_preds))

  in_signal_pad = np.zeros(((n_preds - 1) * hop_size_samples) + win_length_samples)
  # in_signal_pad = np.zeros((n_preds * hop_size_samples + 200 * 220))

  #print('in_signal_pad.shape is {}'.format(in_signal_pad.shape))

  in_signal_pad[0:audio_clip_length_samples] = in_signal

  preds = np.zeros((n_preds, 26, 2))
  mss_in = np.zeros((n_preds, 801, 64))
  events = []

  for i in range(n_preds):
    seg = in_signal_pad[i * hop_size_samples:(i * hop_size_samples) + win_length_samples]
    #print('seg.shape is {}'.format(seg.shape))
    seg = librosa.util.normalize(seg)
    seg_t = librosa.resample(seg, orig_sr=22050, target_sr=16000)
    seg = seg_t

    mss = get_log_melspectrogram(seg)
    M = mss.T
    mss_in[i, :, :] = M

  preds = model.predict(mss_in)
  # preds[:, 0] = (p[:, 0] >= 0.5).astype(np.float)
  # preds[:, 2] = (p[:, 2] >= 0.5).astype(np.float)

  events = []

  for j in range(n_preds):
    p = preds[j, :, :]
    events_curr = []
    win_width = win_length / 26
    for i in range(len(p)):
      if p[i][0] >= 0.5:
        start = win_width * i + win_width * p[i][1]
        end = win_width * i + win_width * p[i][2]
        events_curr.append([start, end, "speech"])

      if p[i][3] >= 0.5:
        start = win_width * i + win_width * p[i][4]
        end = win_width * i + win_width * p[i][5]
        events_curr.append([start, end, "music"])

    se = events_curr
    if j == 0:
      start = 0.0
      end = start + win_length
      if preds.shape[0] > 1:
        end -= discard

      # print("start: {}   end: {}".format(start, end))
    elif j == n_preds - 1:
      start = j * hop_size + discard
      end = start - discard + win_length
      # print("start: {}   end: {}".format(start, end))

    else:
      start = j * hop_size + discard
      end = start + win_length - discard
      # print("start: {}   end: {}".format(start, end))
    
    for k in range(len(se)):
      se[k][0] = max(start, se[k][0] + j * hop_size)
      se[k][1] = min(end, se[k][1] + j * hop_size)

    # print(se)


    for see in se:
     events.append(see) 
    
  # print(events)
  smooth_events = smoothe_events(events)

  return smooth_events

In [ ]:
import time

In [ ]:
start_time = time.time()
see = mk_preds_vector("/content/test_6.wav")
print(see)
stop_time = time.time()
print("Duration: {}".format(stop_time - start_time))

In [ ]:
tt = "/content/test_6.txt"

In [ ]:
see = mk_preds_vector(tt.replace(".txt",".wav"))
n_label = tt.replace(".txt","-se-prediction.txt")

with open(n_label, 'w') as fp:
  fp.write('\n'.join('{},{},{}'.format(round(x[0], 5), round(x[1], 5), x[2]) for x in see))

In [ ]:
# mk_preds_vector("/content/test audio/ROTDevonGoldenHour01022020090000.wav")
start_time = time.time()
see = mk_preds_vector("/content/test audio/ROTDevonGoldenHour01022020090000.wav")
print(see)
stop_time = time.time()
print("Duration: {}".format(stop_time - start_time))

In [ ]:
import time

In [ ]:
import shutil

In [ ]:
def read_annotation(filename):
    events = []
    with open(filename, 'r') as csvfile:
        spamreader = csv.reader(csvfile, delimiter=',', quotechar='|')
        for row in spamreader:
            row[0] = float(row[0])
            row[1] = float(row[1])
            events.append(row)
    return events

In [ ]:
"""
Copy audio files of test set into a separate folder called 'test audio'
"""

audio_files = ['/content/drive/My Drive/MuSpeak/a3-mel-train.npy', '/content/drive/My Drive/MuSpeak/a4-mel-train.npy', '/content/drive/My Drive/BBC RadioMe dataset/ROTDevonGoldenHour28012020090000-mel-train.npy', '/content/drive/My Drive/BBC RadioMe dataset - 2/ROTDevonMorning28012020100000_pt1-mel-train.npy', '/content/drive/My Drive/BBC RadioMe dataset/ROTDevonGoldenHour30012020090000-mel-train.npy', '/content/drive/My Drive/BBC RadioMe dataset/ROTDevonGoldenHour01022020090000-mel-train.npy']

for i in range(len(audio_files)):
  temp = audio_files[i].replace("-mel-train.npy",".txt")
  audio_files[i] = temp  

destination = "/content/test audio/"
try: 
    os.mkdir(destination) 
except OSError as error: 
    pass  


for a in audio_files:
  shutil.copy(a, destination)
  shutil.copy(a.replace(".txt",".wav"), destination)


"""
Smoothe the labels in the .txt files
"""
import csv

destination = "/content/test audio/"
audio_files = glob.glob(destination + "*.txt")

for a in audio_files:
  events = read_annotation(a)
  smooth_events = smoothe_events(events)

  temp_file = a.replace(".txt", "-temp.txt")
  with open(temp_file, 'w', newline='') as csvfile:
      spamwriter = csv.writer(csvfile, delimiter=',',
                              quotechar='|', quoting=csv.QUOTE_MINIMAL)
      for event in smooth_events:
        spamwriter.writerow(event)
  shutil.copyfile(temp_file, a)
  os.remove(temp_file)

In [ ]:
model.load_weights("/content/drive/My Drive/Post-ICASSP 2021/Reliving transitions/Models/CRNN-3/model-best.h5")

In [ ]:
"""
Evaluation on Test set.
"""


# models = glob.glob('/content/drive/My Drive/Experiments/ICASSP 2021/e1/Models/*.h5', recursive = False)

# models = ['/content/drive/My Drive/ICASSP 2021/e1/Models/model 1_3.h5', '/content/drive/My Drive/ICASSP 2021/e1/Models/model 1_4.h5', '/content/drive/My Drive/ICASSP 2021/e1/Models/model 2_4.h5',  '/content/drive/My Drive/ICASSP 2021/e1/Models/model 2_5.h5', '/content/drive/My Drive/ICASSP 2021/e1/Models/model 4_1.h5', '/content/drive/My Drive/ICASSP 2021/e1/Models/model 4_2.h5', '/content/drive/My Drive/ICASSP 2021/e1/Models/model 4_3.h5',  '/content/drive/My Drive/ICASSP 2021/e1/Models/model 4_4.h5',   '/content/drive/My Drive/ICASSP 2021/e1/Models/model 4_5.h5']
# models = ['/content/drive/My Drive/ICASSP 2021/e1/Models/model 102_1.h5']
# models = ['/content/drive/My Drive/Experiments/Models/e2/model48.h5']
# models = ['/content/drive/My Drive/Post-ICASSP 2021/Experiments/Models/model d6_23/model-best.h5']


# models = ['/content/drive/MyDrive/Post-ICASSP 2021/Experiments/5k + 10k comparison/B-GRU - 1/model-best.h5',
#           '/content/drive/MyDrive/Post-ICASSP 2021/Experiments/5k + 10k comparison/B-LSTM - 1/model-best.h5',
#           '/content/drive/MyDrive/Post-ICASSP 2021/Experiments/5k + 10k comparison/CRNN - 1/model-best.h5',
#           '/content/drive/MyDrive/Post-ICASSP 2021/Experiments/5k + 10k comparison/TCN - 1/model-best.h5',
#           '/content/drive/MyDrive/Post-ICASSP 2021/Experiments/5k + 10k comparison/CNN - 1/model-best.h5']



destination = "/content/test audio/"
test_set = glob.glob(destination + "*[0-9].txt")

print(test_set)

eval_path = "/content/drive/My Drive/Post-ICASSP 2021/Reliving transitions/evaluations/eval - test set/"

for tt in test_set:
  # ss, _ = sf.read(tt.replace(".txt",".wav"))
  # oop = mk_preds_fa(tt.replace(".txt",".wav"))

  #print(oop.shape)
  # p_smooth = smooth_output(oop.T, min_speech = 0.8, min_music = 3.4, max_silence_speech = 0.8, max_silence_music = 0.8)
  # p_smooth = p_smooth.T
  see = mk_preds_vector(tt.replace(".txt",".wav"))
  #print(see)
  n_label = tt.replace(".txt","-se-prediction.txt")

  with open(n_label, 'w') as fp:
    fp.write('\n'.join('{},{},{}'.format(round(x[0], 5), round(x[1], 5), x[2]) for x in see))

"""
Code for second round of using sed_eval starts here.. This simply tests on one of the muspeak datasets
"""

se_dir = '/content/drive/My Drive/Data Synthesis/SE Audio Examples'
n = 512

file_list = [
    {
    'reference_file': tt,
    'estimated_file': tt.replace(".txt","-se-prediction.txt")
    }
    for tt in test_set
]

data = []

# Get used event labels
all_data = dcase_util.containers.MetaDataContainer()
for file_pair in file_list:
    reference_event_list = sed_eval.io.load_event_list(
        filename=file_pair['reference_file']
    )
    estimated_event_list = sed_eval.io.load_event_list(
        filename=file_pair['estimated_file']
    )

    data.append({'reference_event_list': reference_event_list,
                'estimated_event_list': estimated_event_list})

    all_data += reference_event_list

event_labels = all_data.unique_event_labels

# Start evaluating

# Create metrics classes, define parameters
segment_based_metrics = sed_eval.sound_event.SegmentBasedMetrics(
    event_label_list=event_labels,
    time_resolution=0.010
)

event_based_metrics = sed_eval.sound_event.EventBasedMetrics(
    event_label_list=event_labels,
    t_collar=0.500
)

# Go through files
for file_pair in data:
    segment_based_metrics.evaluate(
        reference_event_list=file_pair['reference_event_list'],
        estimated_event_list=file_pair['estimated_event_list']
    )

    event_based_metrics.evaluate(
        reference_event_list=file_pair['reference_event_list'],
        estimated_event_list=file_pair['estimated_event_list']
    )

# Get only certain metrics
overall_segment_based_metrics = segment_based_metrics.results_overall_metrics()
print("Accuracy:", overall_segment_based_metrics['accuracy']['accuracy'])

# Or print all metrics as reports

model_basename = "YamNet-23.h5"
seg_eval_basename = "seg eval " + model_basename.replace(".h5", "") + ".txt"
ev_eval_basename = "event eval " + model_basename.replace(".h5", "") + ".txt"
with open(os.path.join(eval_path, seg_eval_basename), mode='w') as fp:
  fp.write(str(segment_based_metrics))

with open(eval_path + "/seg eval " + model_basename.replace(".h5", "") + ".pickle", 'wb') as f:
  pickle.dump(segment_based_metrics, f, pickle.HIGHEST_PROTOCOL)

with open(os.path.join(eval_path, ev_eval_basename), mode = 'w') as fp:
  fp.write(str(event_based_metrics))

with open(eval_path + "/event eval " + model_basename.replace(".h5", "") + ".pickle", 'wb') as f:
  pickle.dump(event_based_metrics, f, pickle.HIGHEST_PROTOCOL)   